In [1]:
import os
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
import pandas as pd
from datetime import datetime
from transformers import (
    TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction,
    InformerConfig, InformerForPrediction
)

os.chdir("../")

from src.data import HF_Dataset
from src.conf import HF_Dataset_Params, TS_Transformer_Params
from src.utils import get_split_limit, transform_start_field
from src.models import TS_Transformer

In [2]:
freq = "1D"

prediction_length = 12

In [3]:
params = HF_Dataset_Params(
    # from DB
    [datetime(2023, 4, 12, 12, 0, 0), datetime(2023, 4, 12, 12, 0, 0)], 
    [[i for i in range(100)], [-i/10 for i in range(100)]],
    [[1], [2]],
    [None, None],
    # from UI
    freq,
    prediction_length
)
custom_ts = HF_Dataset(params)

In [4]:
custom_ts.getDatasetDict()

DatasetDict({
    train: Dataset({
        features: ['start', 'target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'],
        num_rows: 2
    })
    validation: Dataset({
        features: ['start', 'target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'],
        num_rows: 2
    })
    test: Dataset({
        features: ['start', 'target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'],
        num_rows: 2
    })
})

In [5]:
train_dataset = custom_ts.getFormattedDatasetDict()["train"]

test_dataset = custom_ts.getFormattedDatasetDict()["test"]

In [6]:
train_dataset["start"]

[Period('2023-04-12', 'D'), Period('2023-04-12', 'D')]

In [7]:
custom_ts.getFeaturesStats()

{'input_size': 2,
 'num_dynamic_real_features': 0,
 'num_static_categorical_features': 1}

# Test Transformers

In [8]:
transformer_params = TS_Transformer_Params(
    prediction_length, freq, custom_ts.getFeaturesStats(),
    [InformerConfig, InformerForPrediction],
    4, [2], 4, 32, 0.3
)

In [9]:
transformer = TS_Transformer(transformer_params)

transformer._model()

In [10]:
config = transformer.config

In [11]:
model = transformer.model

In [12]:
train_dataloader, test_dataloader = transformer._transformation(train_dataset, test_dataset)

In [13]:
train_dataloader

In [14]:
batch = next(iter(train_dataloader))
for k, v in batch.items():
    print(k, v.shape, v.type())

GluonTSDataError: Input for field "target" does not have the requireddimension (field: target, ndim observed: 1, expected ndim: 2)